In [1]:
# --- Standard library ---
import os
import re
import operator
from typing import TypedDict, Annotated, Literal, Sequence

# --- LangGraph ---
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import tools_condition, ToolNode


# --- LangChain Core ---
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
import requests

# --- LangChain Community ---
from langchain_community.document_loaders import ArxivLoader, PyMuPDFLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS

# --- LangChain OpenAI ---
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# --- LangChain Tools ---
from langchain.tools.retriever import create_retriever_tool

# --- LangChain Hub ---
from langchain import hub

# --- LangChain Text Splitters ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- Other third-party ---
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv()

True

# **Image Weather Tool ( custome built )**

In [3]:
@tool
def analyze_weather_from_image(image_url: str) -> str:
  """
  Analyzes a sky image from a given URL and returns estimated weather conditions.
  """
  llm_image = ChatOpenAI(model="gpt-4o-mini")

  # Pass both text and image content
  response = llm_image.invoke([
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "Analyze this sky image and describe the current weather and image in detail."},
        {"type": "image_url", "image_url": {"url": image_url}}
      ]
    }
  ])

  return response.content


# **Search Tool( pre-built )**

In [4]:
# from langchain_community.tools import DuckDuckGoSearchRun

# search_tool = DuckDuckGoSearchRun()

from langchain_tavily import TavilySearch
search_tool = TavilySearch(max_results = 3)


# **Weather Tool( custome built )**

In [5]:
@tool
def get_weather_data(location: str) -> str:
    """Searches and returns current weather data for the given location."""
    url = f"https://api.weatherstack.com/current?access_key=ec5bb73cd5201057b15ee5d9e78a7c0e&query={location}"
    response = requests.get(url)
    return response.json()

In [6]:
temp = get_weather_data.invoke("lahore")
temp

{'request': {'type': 'City',
  'query': 'Lahore, Pakistan',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Lahore',
  'country': 'Pakistan',
  'region': 'Punjab',
  'lat': '31.550',
  'lon': '74.344',
  'timezone_id': 'Asia/Karachi',
  'localtime': '2025-10-30 17:10',
  'localtime_epoch': 1761844200,
  'utc_offset': '5.0'},
 'current': {'observation_time': '12:10 PM',
  'temperature': 29,
  'weather_code': 143,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
  'weather_descriptions': ['Haze'],
  'astro': {'sunrise': '06:17 AM',
   'sunset': '05:15 PM',
   'moonrise': '01:30 PM',
   'moonset': 'No moonset',
   'moon_phase': 'Waxing Gibbous',
   'moon_illumination': 53},
  'air_quality': {'co': '983.85',
   'no2': '11.95',
   'o3': '249',
   'so2': '27.85',
   'pm2_5': '85.15',
   'pm10': '91.45',
   'us-epa-index': '4',
   'gb-defra-index': '4'},
  'wind_speed': 9,
  'wind_degree': 11,
  'wind_dir': 'NNE',
  'pressure

# **LLM**

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# **Prompt Template( pre-built )**

In [9]:
prompt = hub.pull("hwchase17/react")

# **Agent**

In [10]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data, analyze_weather_from_image],
    prompt=prompt
    )

# **Agent Executor**

In [11]:
agent_executer = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data, analyze_weather_from_image],
    verbose=True)

# **Response**

In [12]:
agent_response = agent_executer.invoke({"input": "Can you tell me the population of Capital of pakistan"})
print(agent_response)



> Entering new AgentExecutor chain...
To find the current population of the capital of Pakistan, which is Islamabad, I will search for the most recent population data. 

Action: tavily_search  
Action Input: "current population of Islamabad Pakistan 2023"  {'query': 'current population of Islamabad Pakistan 2023', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.macrotrends.net/global-metrics/cities/22042/islamabad/population', 'title': 'Islamabad, Pakistan Metro Area Population (1950-2025)', 'content': 'The metro area population of Islamabad in 2023 was 1,232,000, a 2.84% increase from 2022. The metro area population of Islamabad in 2022 was 1,198,000, a 2.92%', 'score': 0.9443876, 'raw_content': None}, {'url': 'https://populationstat.com/pakistan/islamabad', 'title': 'Islamabad, Pakistan Population (2025)', 'content': 'Islamabad Urban Area Population Graph ; 2023, 1,232,000 ; 2024, 1,267,000 ; 2025, 1,301,000 ; 2026, 1,336,000.', 'score': 

In [13]:
agent_response['output']

'The population of Islamabad, the capital of Pakistan, is approximately 1,232,000 in 2023.'

In [14]:
agent_response = agent_executer.invoke({"input": "Analyze this image: https://upload.wikimedia.org/wikipedia/commons/d/d3/July_night_sky_%2835972569256%29.jpg"})
print(agent_response)



> Entering new AgentExecutor chain...
I need to analyze the sky image provided in order to determine the weather conditions depicted in it.  
Action: analyze_weather_from_image  
Action Input: https://upload.wikimedia.org/wikipedia/commons/d/d3/July_night_sky_%2835972569256%29.jpg  The image captures a stunning view of the night sky filled with countless stars and the prominent band of the Milky Way stretching diagonally across the frame. The overall color palette includes deep shades of black and purple, interspersed with white and yellow stars. 

In the foreground, silhouettes of trees are present, adding depth to the scene. The atmosphere appears clear and free of clouds, indicating excellent visibility for stargazing. The tranquil and dark setting suggests that the weather is calm and likely cool, perfect for nighttime observations of celestial phenomena.

The Milky Way is highlighted with soft glows of pink and lavender, suggesting a vibrant density of stars and interstellar dus

In [15]:
agent_response['output']

'The image depicts clear weather conditions with a star-filled night sky, showcasing the Milky Way and providing excellent visibility for stargazing. The atmosphere appears calm and likely cool, suitable for nighttime observations.'